In [1]:
from selenium import webdriver

from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import pprint
import traceback
import os

In [2]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")

In [3]:
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chrome/chromedriver")

/tmp/ipykernel_26556/688692109.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chrome/chromedriver")
/tmp/ipykernel_26556/688692109.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chrome/chromedriver")


In [4]:

from selenium.webdriver.common.keys import Keys

querys = [
    'AI',
    'data scientist',
    'data engineer',
    'data analytic',
    'backend',
    'mobile',
    'devops',
    'tester',
    'qa',
    'software'
]
citys = [
    'All Cities'
]
query = querys[0]
city = citys[0]


In [5]:

sign_in = True
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chrome/chromedriver")

if sign_in:
    url = "https://itviec.com/sign_in"
    driver.get(url)
    web_email = driver.find_element(value='user_email')
    web_email.send_keys('21C29003@student.hcmus.edu.vn')
    web_pass = driver.find_element(value='user_password')
    web_pass.send_keys('ABCdef@123456')
    web_button = driver.find_element(by=By.CLASS_NAME, value='btn-danger')
    time.sleep(1.)
    web_button.submit()
    web_button = driver.find_element(by=By.CLASS_NAME, value = 'swal2-close')
    time.sleep(1.5)
    web_button.click()
    time.sleep(1.5)
else:
    url = "https://itviec.com/"
    driver.get(url)

/tmp/ipykernel_26556/3791082140.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chrome/chromedriver")
/tmp/ipykernel_26556/3791082140.py:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="chrome/chromedriver")


In [7]:

    
    
job_urls = set()

for query in querys:
    if os.path.exists(f'raw_data/{query}.csv'):
        print(f'Done {query}')
        continue
    print(f'Crawl {query} ...')
    url = "https://itviec.com/"
    driver.get(url)

    web_city = driver.find_element(by=By.XPATH, value="//*[@class='search-form__city-section rounded']")
    web_city.click()
    time.sleep(2)
    driver.find_element(value='city-ts-dropdown').find_element(by=By.XPATH, value=f"//div[text()='{city}']").click()

    web_query = driver.find_element(value='query')
    web_query.send_keys(query)
    web_query.send_keys(Keys.ENTER)


    job_infos = []
    while True:
        time.sleep(1)
        web_group_job = driver.find_element(by=By.CLASS_NAME, value='first-group')
        web_jobs = web_group_job.find_elements(By.XPATH, "./div[starts-with(@class, 'job')]")
        k = 0

        time.sleep(1)
        for web_job in web_jobs:
            # try:
                job_url = web_job.find_elements(by=By.CLASS_NAME, value='job-details-link-wrapper')[1].find_element(by=By.XPATH, value='.//a').get_attribute('href')
                # if job_url in job_urls:
                #     continue
                # else: job_urls.add(job_url)

                job_info = {'query':query}
                k += 1
                time.sleep(2.5)
                try:
                    web_job.click()
                except Exception as e:
                    print('web_job click fail')
                    try:
                        web_job.click()
                    except Exception as e:
                        print('web_job click fail2')
                        continue
                    continue
                time.sleep(2.5)

                s = time.time()
                web_job_detail = driver.find_element(by=By.CLASS_NAME, value='job-details--show-employer')
                job_title = web_job_detail.find_element(by=By.CLASS_NAME, value='job-details__title').text
                job_tags = [x.text for x in web_job_detail.find_elements(by=By.CLASS_NAME, value='mkt-track')]
                job_addresses = []
                job_at = None
                last_post = None
                company_name = None


                company_name = driver.find_element(by=By.CLASS_NAME, value='job-details__sub-title').text

                job_addresses_ = web_job_detail.find_elements(by=By.CLASS_NAME, value='svg-icon__text')
                for job_address in job_addresses_:

                    if job_address.find_element(by=By.XPATH, value = '..').get_attribute('class')=='svg-icon__box d-flex align-items-center':
                        job_at = job_address.text
                    elif len(job_address.find_elements(by=By.XPATH, value = '..//*[name()="use" and @*="#location_icon"]')):
                        job_addresses.append(job_address.text[:-8])
                    elif len(job_address.find_elements(by=By.XPATH, value = '..//*[name()="use" and @*="#calendar_icon"]')):
                        last_post = job_address.text
                    else:
                        salary = job_address.text
                        # print(job_address.find_element(by=By.XPATH, value = '..').get_attribute('class'))

                description = None
                reason = None
                skill = None
                why_work=None
                for p in web_job_detail.find_elements(by=By.CLASS_NAME, value='job-details__top-reason-to-join-us') + \
                    web_job_detail.find_elements(by=By.CLASS_NAME, value='job-details__paragraph'):
                    h2 = p.find_element(by=By.XPATH, value='(preceding-sibling::h2)[last()]').text
                    if h2 == 'Top 3 Reasons To Join Us': reason = p.text
                    if h2 == 'Job Description': description = p.text
                    if h2 == 'Your Skills and Experience': skill = p.text
                    if h2 == "Why You'll Love Working Here": why_work = p.text


                company_url = web_job.find_elements(by=By.CLASS_NAME, value='job-details-link-wrapper')[0].find_element(by=By.XPATH, value='.//a').get_attribute('href')

                web_company = driver.find_element(by=By.CLASS_NAME, value='search-page-employer-overview__characteristics')
                company_type = None
                company_people = None
                company_nation = None
                company_working_day = None
                company_ot = None


                company_infos_ = web_company.find_elements(by=By.CLASS_NAME, value='svg-icon__text')
                for company_info in company_infos_:
                    if len(company_info.find_elements(by=By.XPATH, value = '..//*[name()="use" and @*="#setting_icon"]')):
                        company_type = company_info.text
                    elif len(company_info.find_elements(by=By.XPATH, value = '..//*[name()="use" and @*="#people_icon"]')):
                        company_people = company_info.text
                    elif len(company_info.find_elements(by=By.XPATH, value = '..//*[name()="use" and @*="#calendar_icon"]')):
                        company_working_day = company_info.text
                    elif len(company_info.find_elements(by=By.XPATH, value = '..//*[name()="use" and @*="#time_icon"]')):
                        company_ot = company_info.text
                    else :
                        company_nation = company_info.text

                job_info['job_url'] = job_url
                job_info['title'] = job_title
                job_info['tags'] = job_tags
                job_info['address'] = job_addresses
                job_info['at'] = job_at
                job_info['last_post'] = last_post
                job_info['reason'] = reason
                job_info['description'] = description
                job_info['skill'] = skill
                job_info['why_work'] = why_work
                job_info['salary'] = salary
                job_info['company_url'] = company_url
                job_info['company_name'] = company_name
                job_info['company_type'] = company_type
                job_info['company_people'] = company_people
                job_info['company_working_day'] = company_working_day
                job_info['company_ot'] = company_ot
                job_info['company_nation'] = company_nation
                job_infos.append(job_info)
                # break
            # except Exception as e:
            #     print('err')
            #     print(traceback.format_exc())
            #     pass

        # try:
        try:
            time.sleep(2.)
            web_next = driver.find_element(by=By.XPATH, value='//a[@rel="next"]')
            web_next.click()
            time.sleep(2.)
        except Exception as e:
            try:
                print('fail first')
                web_next = driver.find_element(by=By.XPATH, value='//a[@rel="next"]')
                web_next.click()
                time.sleep(2.)
            except Exception as e:
                print('done')
                break
        # except Exception as e:
        #     print('Done')
        #     break
    df = pd.DataFrame(job_infos)
    df.to_csv(f'raw_data/{query}.csv')

Crawl AI ...
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
fail first
done
Done data scientist
Crawl data engineer ...
fail first
done
Crawl data analytic ...
fail first
done
Crawl backend ...
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
fail first
done
Crawl mobile ...
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
web_job click fail
web_job click fail2
fail first
done
Crawl devops ...
web_job click fail
web_job click fail
web_job click fail
web_job click fail
web_job click fail
web_job click fail
fail first
done
Crawl tester ...
web_job click fail
fail first
done
Crawl qa ...
web_job click fail
web_job click fail
fail fir